In [1]:
import ee
import geemap
from geemap.conversion import *

In [17]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
brma2_region = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [23]:
#Calculate NDRE and set palette
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)
ndre_params = ({'min': 0.0,'max': 1.0, 'palette' : 
               ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']})

In [14]:
#Note: To export yearly, I will need to re-define each year
s2 = (ee.ImageCollection('COPERNICUS/S2')
      .filterBounds(brma2_region)
      .filterDate('2016-01-01', '2016-12-31')
      .map(addNDRE).select('NDRE'))

months = ee.List.sequence(1, 12)

In [15]:
# Group by month, and then reduce within groups by median()
# the result is an ImageCollection with one image for each
# month.
def byMonth (m):
    return (s2.filter(ee.Filter.calendarRange(m, m, 'month'))
            .select(0).median()
            .set('month', m))

collection = ee.ImageCollection.fromImages(months.map(byMonth))

In [16]:
count = collection.size().getInfo()
print(count)

12


In [25]:
#Select the first image from the collection (January) to show on the map
Map.addLayer(ee.Image(collection.first()), ndre_params, 'jan ndre')

In [24]:
#Select a specific month to show on the map
month = (collection
         .filterMetadata('month', 'equals', 10) #0 is January, 11 is December
         .select('NDRE'))
Map.addLayer(month, ndre_params, 'nov ndre')